In [1]:
%load_ext autoreload
%autoreload 2

# from helpers.experiments_utils import wrapper
from sympy import *
import numpy as np
import pandas as pd

from other_models.direct_lingam.simulate_data import get_Lambda
from other_models.direct_lingam.DAG import plot_dag
from helpers.models import IV
from helpers.experiments_utils import wrapper, SimulationSettings

# Real Data Cleaning

In [2]:
import os
from sklearn.linear_model import LinearRegression
from helpers.models import CM, ICM

In [3]:
col_names = [
    "ID", "CHAIN", "CO_OWNED", "STATE",
    "SOUTHJ", "CENTRALJ", "NORTHJ", "PA1",
    "PA2", "SHORE",
    "NCALLS", "EMPFT", "EMPPT", "NMGRS",
    "WAGE_ST", "INCTIME", "FIRSTINC",
    "BONUS", "PCTAFF", "MEALS", "OPEN",
    "HRSOPEN", "PSODA", "PFRY", "PENTREE",
    "NREGS", "NREGS11",
    "TYPE2", "STATUS2", "DATE2", "NCALLS2",
    "EMPFT2", "EMPPT2", "NMGRS2", "WAGE_ST2",
    "INCTIME2", "FIRSTIN2", "SPECIAL2", "MEALS2",
    "OPEN2R", "HRSOPEN2", "PSODA2", "PFRY2",
    "PENTREE2", "NREGS2", "NREGS112"
]

col_names_pre = [
    "ID", "CHAIN", "CO_OWNED", "STATE",
    "SOUTHJ", "CENTRALJ", "NORTHJ", "PA1",
    "PA2", "SHORE",
    "EMPFT", "EMPPT",
    "WAGE_ST", "INCTIME", "FIRSTINC",
    "OPEN", "HRSOPEN", "PSODA", "PFRY",
    "PENTREE"
]

col_names_post = [
    "ID", "CHAIN", "CO_OWNED", "STATE",
    "SOUTHJ", "CENTRALJ", "NORTHJ", "PA1",
    "PA2", "SHORE",
    "EMPFT2", "EMPPT2",
    "WAGE_ST2", "INCTIME2", "FIRSTIN2",
    "OPEN2R", "HRSOPEN2", "PSODA2", "PFRY2",
    "PENTREE2"
]

In [4]:
file = "njmin/public.dat"
file_path = os.path.join(os.getcwd(), file)
df = pd.read_table(file_path, delim_whitespace=True, names=col_names, na_values='.')

/var/folders/nd/nn8th6r16cq6sytrzw1nw_9w0000gn/T/ipykernel_925/60473316.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(file_path, delim_whitespace=True, names=col_names, na_values='.')


In [5]:
data_pre = df[col_names_pre].copy()
data_post = df[col_names_post].copy()
data_post.columns = data_pre.columns
rate = 0.5

tmp = len(data_pre)*[0.]
data_pre.insert(1, "Treatment", tmp)
data_pre["Empl"] = data_pre["EMPFT"] + data_pre["EMPPT"]*rate

tmp = len(data_post)*[1.]
data_post.insert(1, "Treatment", tmp)
data_post["Empl"] = data_post["EMPFT"] + data_post["EMPPT"]*rate

data_pre["PostTreatment"] = data_pre["STATE"]*data_pre["Treatment"]
data_post["PostTreatment"] = data_post["STATE"]*data_post["Treatment"]

mask = data_post["Empl"]==0
mask = ~mask
data_pre = data_pre[mask]
data_post = data_post[mask]

In [6]:
data_pre_new = data_pre.drop(["ID", "CHAIN", "EMPFT", "EMPPT"], axis=1)
data_post_new = data_post.drop(["ID", "CHAIN", "EMPFT", "EMPPT"], axis=1)
mask = data_pre_new.isnull().any(axis=1) | data_post_new.isnull().any(axis=1)
mask = ~mask

data_pre_new = data_pre_new[mask]
data_post_new = data_post_new[mask]

## Results

In [ ]:
from joblib import Parallel, delayed
data = pd.concat([data_pre_new, data_post_new], ignore_index=True)
X = data.drop(["Empl"], axis=1).to_numpy()
Y = data["Empl"].to_numpy()

reg = LinearRegression().fit(X, Y)
reg.score(X, Y)
beta = reg.coef_[-1]
print("Beta estimated by TWFE: ", beta)

Beta estimated by TWFE:  2.6843899743729382


In [14]:
reg = LinearRegression().fit(X, Y)
Z = data_pre_new["Empl"].to_numpy() - np.matmul(X[:len(data_pre_new), :-1], reg.coef_[:-1])
X_new = data_pre_new["STATE"].to_numpy(dtype=np.float32)
Y = data_post_new["Empl"].to_numpy() - np.matmul(X[len(data_pre_new):, :-1], reg.coef_[:-1])

Z -= np.mean(Z)
X_new -= np.mean(X_new)
Y -= np.mean(Y)

# High-order method below

In [15]:
highest_l = 1
data_zxy = np.vstack([Z, X_new, Y]).transpose()
model = CM(data_zxy, highest_l=highest_l)
estimate = model.estimate_effect()
estimate_cross_moment = model.estimate_effect_cross_moment()
estimate_grica = model.estimate_effect_grica()
print("Estimate:", estimate)
print("Estimate cross moment:", estimate_cross_moment)
print("Estimate grica:", estimate_grica)

Estimate: 2.6843899743731168
Estimate cross moment: 2.6843899743729405
Estimate grica: 1.9745572805404663


In [18]:
highest_l = 2
data_zxy = np.vstack([Z, X_new, Y]).transpose()
model = CM(data_zxy, highest_l=highest_l)
estimate = model.estimate_effect()
estimate_cross_moment = model.estimate_effect_cross_moment()
estimate_grica = model.estimate_effect_grica()
print("Estimate:", estimate)
print("Estimate cross moment:", estimate_cross_moment)
print("Estimate grica:", estimate_grica)

Estimate: 2.719978435502854
Estimate cross moment: 2.6843899743729405
Estimate grica: 4.167938709259033


In [19]:
highest_l = 1
data_zxy = np.vstack([Z, X_new, Y]).transpose()
model = ICM(data_zxy, highest_l=highest_l)
estimate = model.estimate_effect()
estimate_cross_moment = model.estimate_effect_cross_moment()
estimate_effect_minimization = model.estimate_effect_minimization()
estimate_grica = model.estimate_effect_grica()
print("Estimate:", estimate)
print("Estimate cross moment:", estimate_cross_moment)
print("Estimate effect minimization:", estimate_effect_minimization)
print("Estimate grica:", estimate_grica)

Estimate: 8.26289653667604
Estimate cross moment: 2.6843899743729405
Estimate effect minimization: 9.406543873074266
Estimate grica: 2.854421615600586


In [21]:
highest_l = 2
data_zxy = np.vstack([Z, X_new, Y]).transpose()
model = ICM(data_zxy, highest_l=highest_l)
estimate = model.estimate_effect()
estimate_grica = model.estimate_effect_grica()
print("Estimate:", estimate)
print("Estimate grica:", estimate_grica)

Estimate: 3.008679290461188
Estimate grica: 3.508751392364502
